In [101]:
import psycopg2
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [8]:
conn = psycopg2.connect(dbname='maryblack', user='maryblack', 
                        password='maryblack', host='localhost')

cursor = conn.cursor()

## Import testset_B.tsv into the SQL DB

In [32]:
create_table = """
--DROP TABLE TESTSET_B;
CREATE TABLE TESTSET_B (
    PRODUCTID VARCHAR(32),
    BRAND VARCHAR(32),
    RAM_GB NUMERIC,
    HDD_GB NUMERIC,
    GHZ NUMERIC,
    PRICE NUMERIC
);

COPY TESTSET_B(PRODUCTID, BRAND, RAM_GB, HDD_GB, GHZ, PRICE)
FROM '/tmp/testset_B.tsv'
DELIMITER E'\t'
CSV HEADER;
"""
cursor.execute(create_table)

In [66]:
df = pd.read_sql('select * from testset_B;', conn)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [104]:
df

,productid,brand,ram_gb,hdd_gb,ghz,price
0,000394,Samsung,16.0,1000.0,3.2,90900.0
1,000828,Lenovo,8.0,2000.0,2.5,51900.0
2,000995,Acer,4.0,1000.0,2.3,44900.0
3,001009,HP,4.0,500.0,2.3,42900.0
4,001093,Acer,8.0,2000.0,2.3,58900.0
...,...,...,...,...,...,...
6484,998890,Apple,4.0,2000.0,1.8,106900.0
6485,998992,asus,16.0,500.0,3.0,67900.0
6486,999168,Acer,8.0,2000.0,2.3,49900.0
6487,999618,Apple,4.0,1000.0,1.8,97900.0


In [92]:
# Ranks based on column “Price”, grouped by column “brand”
sql_a = """
SELECT
    productid,
    brand,
    price,
    hdd_gb,
    ghz,
    RANK () OVER ( 
        PARTITION BY brand
        ORDER BY price DESC
    ) price_rank 
FROM testset_B
"""

# Minimum and maximum of column “HDD_GB”
sql_b = """
SELECT
    min(hdd_gb) min_hdd_gb,
    max(hdd_gb) max_hdd_gb
FROM testset_B
"""

# Median of column “GHz”, grouped by column “RAM_GB”
# since median is the 50th percentile, we can use it as a proxy to median
sql_c = """
SELECT
    ram_gb, 
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY GHZ) median_ghz
FROM testset_B
group by ram_gb
"""

In [102]:
pd.read_sql(sql_b, conn).to_csv("report_a.csv", index=False, sep=";")

In [103]:
# and check with pandas
df.groupby("ram_gb").agg({"ghz": "median"})

,ghz
ram_gb,
4.0,2.5
8.0,2.5
16.0,2.5
